In [1]:
import requests
import psycopg2
from datetime import datetime, timedelta
import time

In [2]:
start_date = datetime.strptime("2023-07-17", "%Y-%m-%d")
end_date = datetime.strptime("2023-07-17", "%Y-%m-%d")

date_list = []
current_date = start_date

while current_date <= end_date:
    formatted_date = current_date.strftime("%Y-%m-%d")
    date_list.append(formatted_date)
    current_date += timedelta(days=1)


In [3]:
job_list = ['arrange', 'in', 'out', 'retin', 'retout', 'trans', 'SHOP_REQ']
# arrange 조정
# in 입고
# out 출고
# retin 반품입고
# retout 반품출고
# trans 배송처리
# SHOP_REQ 매장출고

In [4]:
def get_stock_log(start_date, end_date, job, page):


    partner_key = '73c6ac2e3366905793ab35d5a222d572'
    domain_key = '73c6ac2e33669057718abd505d76aa87366d3eed6643997457a0f9a03797aa9f'
    # start_date = '2023-01-01'
    # end_date = '2023-05-21'
    # job = 'in'
    limit = '100'
    

    url = f'http://api2.cloud.ezadmin.co.kr/ezadmin/function.php?partner_key={partner_key}&domain_key={domain_key}&action=get_stock_tx_info&start_date={start_date}&end_date={end_date}&job={job}&page={page}&limit={limit}'
    res = requests.get(url)

    if res.status_code == 200:
        return res.json()['data']
    else:
        return ValueError


In [5]:
def save_stock_log(cursor, data_list):

    query = """
        INSERT INTO "stock_log" (seq, crdate, product_id, job, qty, stock, bad, owner, memo, supply_code, supply_price, name, options, barcode, origin, brand, enable_sale, org_price, org_price_qty, order_seq, sheet, sheet_name, product_memo) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
        ON CONFLICT (seq) DO UPDATE 
        SET crdate = EXCLUDED.crdate, product_id = EXCLUDED.product_id, job = EXCLUDED.job, qty = EXCLUDED.qty, stock = EXCLUDED.stock, 
            bad = EXCLUDED.bad, owner = EXCLUDED.owner, memo = EXCLUDED.memo, supply_code = EXCLUDED.supply_code, 
            supply_price = EXCLUDED.supply_price, name = EXCLUDED.name, options = EXCLUDED.options, barcode = EXCLUDED.barcode, 
            origin = EXCLUDED.origin, brand = EXCLUDED.brand, enable_sale = EXCLUDED.enable_sale, org_price = EXCLUDED.org_price, 
            org_price_qty = EXCLUDED.org_price_qty, order_seq = EXCLUDED.order_seq, sheet = EXCLUDED.sheet, 
            sheet_name = EXCLUDED.sheet_name, product_memo = EXCLUDED.product_memo;
    """

    for d in data_list:
        cursor.execute(query, d)


In [6]:
conn = psycopg2.connect(host='115.68.228.168', user='postgres', password='cldngks', dbname='phytogether', port=5432)
cursor = conn.cursor()


for date in date_list:
    for job in job_list:
        page = 1
        data = ['init']
        while len(data) > 0:
            data = get_stock_log(date, date, job, str(page))
            data_list = [list(d.values()) for d in data]
            save_stock_log(cursor, data_list)
            # time.sleep(3)
            print(date + ' | ' + job + ' | ' + str(page))
            page += 1


conn.commit()
conn.close()

2023-07-17 | arrange | 1
2023-07-17 | in | 1
2023-07-17 | out | 1
2023-07-17 | retin | 1
2023-07-17 | retout | 1
2023-07-17 | trans | 1
2023-07-17 | trans | 2
2023-07-17 | trans | 3
2023-07-17 | trans | 4
2023-07-17 | trans | 5
2023-07-17 | trans | 6
2023-07-17 | trans | 7
2023-07-17 | trans | 8
2023-07-17 | trans | 9
2023-07-17 | trans | 10
2023-07-17 | trans | 11
2023-07-17 | SHOP_REQ | 1
